In [10]:
ShmekelPath = r'C:\Users\edenp\PycharmProjects\Shmekel'
template_path = r'configs/eden/test/WidthDepthGS6.json'
configs_dir = r'configs/eden/test/depth_width_grid_search_5'

import os
os.chdir(ShmekelPath)

import pandas
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
from pandas.api.types import is_string_dtype, is_numeric_dtype
import numpy as np
from api.core.grid_search import GridSearch


from Utils.logger import logger
logger.set_stream_level('warning')


gs = GridSearch(template_path=template_path,configs_dir=configs_dir)
THE_METRIC = 'acc'


def _generate_plotly_dim_dict(df: pandas.DataFrame, field):
    dim_dict = {}
    dim_dict["label"] = field
    column = df[field]
    if is_numeric_dtype(column):
        dim_dict["values"] = column
    elif is_string_dtype(column):
        texts = column.unique()
        dim_dict["values"] = [
            np.argwhere(texts == x).flatten()[0] for x in column
        ]
        dim_dict["tickvals"] = list(range(len(texts)))
        dim_dict["ticktext"] = texts
    else:
        raise Exception("Unidentifiable Type")

    return dim_dict


def data_frame_to_parallel_plot_data(df: pandas.DataFrame):
    line = dict(color=-df[THE_METRIC], colorscale='Jet')
    dimensions = [_generate_plotly_dim_dict(df, field) for field in df]

    return [go.Parcoords(line=line, dimensions=dimensions)]


def grid_search_to_data_frame(gs: GridSearch, the_metric=THE_METRIC, metrics=('val_acc',)):
    converter = gs.get_exp_name_2_combs_converter(minimal=True)

    columns = None
    for comb in converter.values():
        columns = [field for field in comb.keys()]
        break

    columns = ('__Name__',) +  tuple([c for c in columns if c not in ['__Name__']]) + tuple(metrics) + (the_metric,)

    df = {field: [] for field in columns}
    for exp in gs:
        line = converter[exp.name]
        
        epoch = exp.results.get_best_epoch(the_metric)
        print(exp.name)
        epoch.print()
        #epoch = exp.results.epoch_list[3]
        #print(exp.name, epoch['val_uncertain_fraction'])

        for m in metrics + (the_metric,):
            if m == 'runtime':
                line[m] = exp.get_avg_runtime()
            else:
                line[m] = epoch[m]

        for field, val in line.items():
            if isinstance(val, list):
                val = str(val)
            
            df[field].append(val)
    
#     _ = df.pop('__InputShape__')
    
    return pandas.DataFrame.from_dict(df)

In [11]:
plotly.offline.iplot(data_frame_to_parallel_plot_data(grid_search_to_data_frame(gs)))

2020-08-26 20:43:19,833 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:19,888 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:19,904 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:19,954 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:19,971 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,028 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,048 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead


depth_1--width_128
Epoch 724 
--------------------------------------------------
--> val_loss ...................... 0.695
--> val_acc ....................... 0.513
--> loss .......................... 0.69
--> acc ........................... 0.531
depth_1--width_32
Epoch 732 
--------------------------------------------------
--> val_loss ...................... 0.693
--> val_acc ....................... 0.512
--> loss .......................... 0.691
--> acc ........................... 0.527
depth_1--width_64
Epoch 750 
--------------------------------------------------
--> val_loss ...................... 0.694
--> val_acc ....................... 0.513
--> loss .......................... 0.69
--> acc ........................... 0.529


2020-08-26 20:43:20,123 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,141 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,214 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,232 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,306 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead


depth_2--width_128
Epoch 736 
--------------------------------------------------
--> val_loss ...................... 0.736
--> val_acc ....................... 0.506
--> loss .......................... 0.654
--> acc ........................... 0.596
depth_2--width_32
Epoch 706 
--------------------------------------------------
--> val_loss ...................... 0.696
--> val_acc ....................... 0.512
--> loss .......................... 0.688
--> acc ........................... 0.538
depth_2--width_64

2020-08-26 20:43:20,318 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,404 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,420 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,511 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead


Epoch 722 
--------------------------------------------------
--> val_loss ...................... 0.704
--> val_acc ....................... 0.507
--> loss .......................... 0.678
--> acc ........................... 0.56
depth_3--width_128
Epoch 747 
--------------------------------------------------
--> val_loss ...................... 0.919
--> val_acc ....................... 0.505
--> loss .......................... 0.583
--> acc ........................... 0.655



2020-08-26 20:43:20,529 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead
2020-08-26 20:43:20,623 default_logger WARNING  parameter "feature_list" is deprecated. use "input_features" instead


depth_3--width_32
Epoch 747 
--------------------------------------------------
--> val_loss ...................... 0.7
--> val_acc ....................... 0.509
--> loss .......................... 0.683
--> acc ........................... 0.547
depth_3--width_64
Epoch 723 
--------------------------------------------------
--> val_loss ...................... 0.726
--> val_acc ....................... 0.508
--> loss .......................... 0.661
--> acc ........................... 0.585


In [21]:
list(gs.iter_exps())[0].results.history.keys()

JSONDecodeError: Expecting value: line 17 column 18 (char 512)